# Multisample analysis

### Setup

In [ ]:
import missionbio.mosaic as ms
from IPython.display import display, HTML

In [ ]:
# Load a multi-sample file
# By default, a multi-sample h5 file loads as a SampleGroup object

group = ms.load_example_dataset("Multisample PBMC")

In [ ]:
# To analyze one sample at a time access, them using the samples attribute
sample = group.samples[1]

# These are the samples in the h5 file
[s.name for s in group.samples]

### Applying functions

In [ ]:
# To apply a function across all samples use the `apply` method on the SampleGroup
# It returns a list of returned objects for each sample

def filt(sample):
    filt_vars = sample.dna.filter_variants()
    return filt_vars

filtered_variants = group.apply(filt)
filtered_variants

In [ ]:
# Subset the same variants in all dna assays
# It is important to maintain the same variants across all dna assays

og_num_vars = [s.dna.shape[1] for s in group.samples]

var_union = list(set().union(*filtered_variants))
for sample in group:
    sample.dna = sample.dna[:, var_union]  # Subsets all samples with the same variants

new_num_vars = [s.dna.shape[1] for s in group.samples]

print(og_num_vars, new_num_vars)  # Thee old and new number of variants for each sample in the group

In [ ]:
# The functions applied on each sample can be more complex - like this assignment and relabeling method
# Note the original labels can be uncoordinated across samples in the group
# The labels are changed to ensure that each label is for the same clone

variants_of_interest = ['chr7:148508833:A/G', 'chr17:29559932:C/A', 'chr4:55599436:T/C']
def cluster(sample):
    clone_table = sample.dna.group_by_genotype(variants_of_interest, max_ado_score=0.8)

    # Rename labels so that each sample has the same labels
    # Here the signature of each variant is used to rename the labels
    df = sample.dna.signature("NGT").loc[:, variants_of_interest]
    names = df.apply(lambda vs: "-".join([str(int(v)) for v in vs]), axis=1)
    label_map = {i: n for i, n in names.items()}
    
    # Don't rename the outlier categories:
    for lab in ["missing", "small", "ADO"]:
        del label_map[lab]

    sample.dna.rename_labels(label_map)
    clone_table = clone_table.rename(index=label_map)

    return clone_table  # Return the clone table

tables = group.apply(cluster)

for t in tables:  # The clone tables for each sample
    display(HTML(t.to_html()))

### Drawing figures

In [ ]:
# Displaying the same plot across all the samples

for sample in group:
    sample.dna.heatmap("NGT_FILTERED", features=variants_of_interest).show()

In [ ]:
# Normalize Protein and look for patterns

for sample in group:
    sample.protein.normalize_reads()
    sample.heatmap(("dna", "protein")).show()

In [ ]:
variants_of_interest = ['chr17:29559932:C/A', 'chr4:55599436:T/C', 'chr7:148508833:A/G']
proteins_of_interest = ['CD19', "CD34", "CD30"]
clones_of_interest = ["0-1-1", "1-1-2"]

for sample in group:
    s = sample[sample.dna.barcodes(clones_of_interest)]
    s.dna = s.dna[:, variants_of_interest]
    s.protein = s.protein[:, proteins_of_interest]
    s.clone_vs_analyte("protein")

### Multisample plots

In [ ]:
# Draw a fishplot for the dna labels
# From the proportions in the heatmaps - The two clones of interest are 0-1-1 and 2-20

group.fishplot(
    "dna",
    sample_order=["Sample 1", "Sample 2"],
    labels=["0-1-1", "1-1-2"],
    parents=[None, None]
)

In [ ]:
# Draw a barplot for the dna labels

group.barplot(
    "dna",
    sample_order=["Sample 1", "Sample 2"],
    label_order=["0-1-1", "1-1-2"],
    percentage=True
)